In [6]:
%load_ext autoreload
%autoreload 2

from datetime import datetime

import pandas as pd

from short_tracker.data import (
    query_all_sec_metadata, query_uk_si_disclosures,
    SHORT_URL_UK, DATE_COL, FUND_COL, ISIN_COL, SHORT_POS_COL, SHARE_ISSUER_COL,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
date_now = datetime.today().date()
# exp_upd_time = datetime.combine(date_now, UK_MKT_EARLY_CLOSE)
# exp_upd_time

discl_data, rept_date = query_uk_si_disclosures(SHORT_URL_UK) # exp_upd_time
rept_date

datetime.date(2022, 12, 23)

In [12]:
cur_discl = discl_data['current']
hist_discl = discl_data['historic']
all_isins = pd.concat([cur_discl[ISIN_COL], hist_discl[ISIN_COL]]).unique()

In [13]:
total_shorts = cur_discl.groupby(SHARE_ISSUER_COL)[SHORT_POS_COL].agg(['sum', 'count'])

In [16]:
total_shorts.sort_values(by='sum', ascending=False)

,sum,count
Name of Share Issuer,,
ASOS PLC,7.42,5
ITM POWER PLC,5.93,6
NAKED WINES PLC,5.79,4
KINGFISHER PLC,5.64,4
BOOHOO GROUP PLC,5.18,6
...,...,...
SCHRODERS PLC,0.50,1
AVACTA GROUP PLC,0.50,1
RENTOKIL INITIAL PLC,0.50,1


In [18]:
cur_discl[cur_discl[DATE_COL]=='2022-12-22']

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,0.96,2022-12-22
16,Citadel Advisors LLC,ASOS PLC,GB0030927254,0.81,2022-12-22
46,Millennium International Management LP,BRITISH LAND CO PLC,GB0001367019,0.59,2022-12-22
53,Tellworth Investments LLP,CARNIVAL PLC,GB0031215220,1.20,2022-12-22
101,GLG Partners LP,Genus plc,GB0002074580,0.50,2022-12-22
104,WorldQuant LLC,HALFORDS GROUP PLC,GB00B012TP20,0.61,2022-12-22
172,WorldQuant LLC,Moonpig Group PLC,GB00BMT9K014,0.58,2022-12-22
182,Tellworth Investments LLP,NATIONAL EXPRESS GROUP PLC,GB0006215205,0.81,2022-12-22
204,Astaris Capital Management LLP,PETROFAC LTD,GB00B0H2K534,1.11,2022-12-22
242,GLG Partners LP,SERCO GROUP PLC,GB0007973794,0.70,2022-12-22


In [35]:
cur_discl_ind = cur_discl.set_index([ISIN_COL, FUND_COL]).index
hist_discl_ind = hist_discl.set_index([ISIN_COL, FUND_COL]).index

In [65]:
# dupl_discl = hist_discl[]
dupl_discl_ind = hist_discl.duplicated(subset=[FUND_COL, ISIN_COL, DATE_COL], keep=False)
dupl_discl_ind_ = hist_discl.loc[dupl_discl_ind].set_index([ISIN_COL, FUND_COL]).index.drop_duplicates()
hist_discl.loc[dupl_discl_ind]

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
23713,Bercheva Opportunities Ltd,EQTEC PLC,IE00BH3XCL94,1.41,2018-03-28
23714,Bercheva Opportunities Ltd,EQTEC PLC,IE00BH3XCL94,2.19,2018-03-28
46085,"Jane Street Group, LLC",MICRO FOCUS INTERNATIONAL,GB00BJ1F4N75,0.64,2017-09-01
46086,"Jane Street Group, LLC",MICRO FOCUS INTERNATIONAL,GB00BJ1F4N75,0.00,2017-09-01
57939,Roble S.L.,QUINDELL PORTFOLIO PLC,GB00BMTS9H89,0.54,2014-02-28
57940,Roble S.L.,QUINDELL PORTFOLIO PLC,GB00BMTS9H89,0.61,2014-02-28


In [67]:
hist_discl.set_index([ISIN_COL, FUND_COL]).loc[dupl_discl_ind_].reset_index().sort_values(by=[FUND_COL, ISIN_COL, DATE_COL, SHORT_POS_COL]).to_clipboard()

In [68]:
def pivot_discl(sec_discl_df):
    return sec_discl_df.pivot_table(index=DATE_COL, columns=FUND_COL, values=SHORT_POS_COL, aggfunc='max')

hist_discl_dict = {isin: pivot_discl(df) for isin, df in hist_discl.groupby(ISIN_COL)}

In [70]:
hist_discl_dict['GB00BJ1F4N75']

Position Holder,AKO Capital LLP,"AQR Capital Management, LLC",Ardevora Asset Management LLP,"BlackRock Institutional Trust Company, National Association",BlackRock Investment Management (UK) Limited,CapeView Capital LLP,"Coltrane Asset Management, L.P.",D. E. Shaw & Co. LP,Gladstone Capital Management LLP,"Glazer Capital, LLC",...,Marshall Wace LLP,Millennium International Management LP,Moore Europe Capital Management LLP,Naya Capital Management UK Limited,Numeric Investors LLC,Sculptor Capital LP,Starboard Value LP,Sylebra Capital Limited,Third Point LLC,WorldQuant LLC
Position Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-24,NaN,NaN,NaN,0.49,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-28,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-30,NaN,NaN,NaN,0.45,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-22,NaN,NaN,NaN,NaN,0.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58,NaN,NaN
2022-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.47,NaN,NaN
2022-11-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [289]:
l = ['GB00BF8Q6K64', 'GB0001367019', 'GB0033057794', 'GB00BF8Q6K64']
d, err = query_all_sec_metadata(l, "ID_ISIN")

In [297]:
exch_codes = {k: [x['exchCode'] for x in v] for k, v in d.items()}

In [143]:
def remove_dupl_shorts(discl_df : pd.DataFrame) -> pd.DataFrame:
    """Checks for duplicate disclosures per fund + date + isin.
    If duplicate is found:
    - silently drop if the net short values are equal
    - otherwise log the duplicated rows and take the first one

    Args:
        discl_df: a df of disclosures per date + fund + isin

    Returns:
        pd.DataFrame: of the input disclosures but with duplicates removed
        as described above
        
    Warning: doesn't detect duplicates from mismatching isins (though this is an unlikely
    edge case...)
    """
    discl_df_ = discl_df.drop_duplicates()
    prim_key_cols = [DATE_COL, FUND_COL, ISIN_COL]
    
    dupl_rows = discl_df_.duplicated(subset=prim_key_cols, keep='first')
    num_dupl_rows = dupl_rows.sum()
    
    if num_dupl_rows:
        logger.warning(
            f"Found {num_dupl_rows} duplicated rows: "
            f"{discl_df_.loc[dupl_rows].to_string()}"
        )
    return discl_df_.loc[~dupl_rows]


def check_cur_hist_discl_overlap(cur_discl, hist_discl):
    """Check if the reported current disclosed positions overlap with
    any of the historical positions.
    """
    idx = [DATE_COL, FUND_COL, ISIN_COL]
    overlap_ind = cur_discl.set_index(idx).index.isin(hist_discl.set_index(idx).index)

In [259]:
sec_metadata, err_isins = query_all_sec_metadata(all_isins, "LN", "ID_ISIN")
print(len(err_isins))
pd.DataFrame(sec_metadata)

No data for id GB0006834344, error: None
No data for id IM00B8BV8G91, error: None
No data for id JP3325200008, error: None
No data for id GB00BKDM7X41, error: None
No data for id GB00BVFD7Q58, error: None
No data for id GB00B06GM726, error: None
No data for id GB00B09LQS34, error: None
No data for id GB00BMSKPJ95, error: None
No data for id GB00B6774699, error: None
No data for id GB00BLP5YB54, error: None
No data for id GB0000031285, error: None
No data for id GB00BMCL8X32, error: None
No data for id GB00B61D2N63, error: None
No data for id AEDFXA1EN018, error: None
No data for id GB00B0672758, error: None
No data for id IM00B39HQT38, error: None
No data for id BMG0114P1005, error: None
No data for id GB00B2QMX606, error: None
No data for id GB00BK1PTB77, error: None
No data for id GB00BQQMCJ47, error: None
No data for id GB00BQ1XTV39, error: None
No data for id GB00BLRLH124, error: None
No data for id GB0000282623, error: None
No data for id GB0032087826, error: None
No data for id G

78


,GB00BF8Q6K64,GB00B18S7B29,GB00BNDRMJ14,GB00B1XZS820,GB00BJTNFH41,GB00BYM8GJ06,GB00B132NW22,GB0030927254,GB00BVGBWW93,GB00BN7CG237,...,GB00BVG6X439,JE00B6Y3DV84,GB00BD6RF223,GB00B2PDGW16,GB00B1KJJ408,GB00BL6C2002,GB0009692319,GB00BL9YR756,GB00BVG1CF25,GB0001570810
figi,BBG000N2C023,BBG000QWQ5W5,BBG010K53NF4,BBG000BWF7M0,BBG005ZD5X21,BBG00BWCTGG9,BBG000BQY7Z8,BBG000KL73H8,BBG000CPPKR9,BBG00LY4RF68,...,BBG0086C7Q77,BBG002ZPXXB0,BBG00CFWZ1D1,BBG000MLFFM4,BBG000BRVH05,BBG00ZVX07Y2,BBG000FZGLX5,BBG011PF19N9,BBG0082YW401,BBG000BVKQF2
name,ABRDN PLC,AFC ENERGY PLC,ALPHAWAVE IP GROUP PLC,ANGLO AMERICAN PLC,AO WORLD PLC,ASCENTIAL PLC,ASHMORE GROUP PLC,ASOS PLC,ASSURA PLC,ASTON MARTIN LAGONDA GLOBAL,...,VPC SPECIALTY LENDING INVEST,WANDISCO PLC,WATKIN JONES PLC,WH SMITH PLC,WHITBREAD PLC,WICKES GROUP PLC,WILMINGTON PLC,WISE PLC - A,SCHRODER UK PUBLIC PRIVATE,XAAR PLC
ticker,ABDN,AFC,AWE,AAL,AO/,ASCL,ASHM,ASC,AGR,AML,...,VSL,WAND,WJG,SMWH,WTB,WIX,WIL,WISE,SUPP,XAR
exchCode,LN,LN,LN,LN,LN,LN,LN,LN,LN,LN,...,LN,LN,LN,LN,LN,LN,LN,LN,LN,LN
compositeFIGI,BBG000N2BZ81,BBG000QWQ5M6,BBG010K53ND6,BBG000BWF7C1,BBG000NKL498,BBG00BWCTGF0,BBG000BQY715,BBG000KL72Y1,BBG000CPPKH0,BBG00LY4RF59,...,BBG0086C7Q68,BBG002ZPXX75,BBG00CFWZ169,BBG000MLFDZ5,BBG000BRVG61,BBG00ZVX0790,BBG000FZGKX7,BBG011PF19M0,BBG0082YW3Z5,BBG000BVKQ53
securityType,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,REIT,Common Stock,...,Closed-End Fund,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Closed-End Fund,Common Stock
marketSector,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,...,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity,Equity
shareClassFIGI,BBG001SGL070,BBG001SS0FJ1,BBG010K53NJ0,BBG001S5Z1X8,BBG001T56Q82,BBG00BWCTGL3,BBG001SGLYM9,BBG001SJNFF0,BBG001SFN828,BBG00LY4RF95,...,BBG0086C7QB2,BBG002ZPXXM8,BBG00CFWZ150,BBG001SN9TC0,BBG001S6NGL9,BBG00ZVX07B7,BBG001S8PH77,BBG011PF19T3,BBG0082YW438,BBG001SBB0X3
securityType2,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,REIT,Common Stock,...,Mutual Fund,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Common Stock,Mutual Fund,Common Stock
securityDescription,ABDN,AFC,AWE,AAL,AO/,ASCL,ASHM,ASC,AGR,AML,...,VSL,WAND,WJG,SMWH,WTB,WIX,WIL,WISE,SUPP,XAR


In [283]:
cur_discl

,Position Date,Position Holder,ISIN,Net Short Position (%)
0,2022-12-16,BlackRock Investment Management (UK) Limited,GB00BF8Q6K64,1.05
1,2022-11-22,Citadel Advisors Europe Limited,GB00BF8Q6K64,0.90
2,2022-12-22,Citadel Advisors LLC,GB00BF8Q6K64,0.96
3,2022-12-16,GLG Partners LP,GB00BF8Q6K64,1.15
4,2022-12-16,Qube Research & Technologies Limited,GB00BF8Q6K64,0.55
...,...,...,...,...
291,2022-11-11,GLG Partners LP,GB00B5N0P849,0.68
292,2022-12-16,Qube Research & Technologies Limited,GB00B5N0P849,0.81
293,2022-12-13,Marshall Wace LLP,JE00B8KF9B49,0.79
294,2022-11-10,GLG Partners LP,SG9999003735,1.06


In [152]:
hist_fund_inst_shorts = pd.Index(hist_discl[[FUND_COL, ISIN_COL]].drop_duplicates())
cur_fund_inst_shorts = pd.Index(cur_discl[[FUND_COL, ISIN_COL]]) # FIXME: check for dupes

In [153]:
existing_shorts = cur_fund_inst_shorts.isin(hist_fund_inst_shorts)

cur_fund_inst_shorts_ = pd.concat([cur_discl[existing_shorts], hist_discl]) # series of shorts leading to current short
existing_fund_inst_shorts = cur_discl[~existing_shorts] # new shorts that haven't historically changed

In [148]:
a = cur_fund_inst_shorts_[(cur_fund_inst_shorts_[FUND_COL]=='Sandbar Asset Management LLP')&(cur_fund_inst_shorts_[ISIN_COL]=='GB00B7KR2P84')]

In [149]:
a[a[DATE_COL].duplicated(keep=False)]

,Position Date,Position Holder,ISIN,Net Short Position (%)
22581,2022-09-01,Sandbar Asset Management LLP,GB00B7KR2P84,2.21
22582,2022-09-01,Sandbar Asset Management LLP,GB00B7KR2P84,2.21


In [156]:
def _reindex_dates(df):
    tgt_index = pd.bdate_range(df[DATE_COL].min(), df[DATE_COL].max(), name=DATE_COL)
    return df.set_index([DATE_COL]).reindex(tgt_index, method="ffill").reset_index()

cur_fund_inst_shorts_ffill = cur_fund_inst_shorts_.groupby([ISIN_COL, FUND_COL], as_index=False).apply(_reindex_dates).reset_index(drop=True)

ValueError: cannot reindex a non-unique index with a method or limit

In [8]:
cur_discl.groupby('ISIN')['Net Short Position (%)'].sum().sort_values()[::-1]

ISIN
GB0030927254    7.50
GB00B0130H42    5.88
GB00B021F836    5.79
GB0033195214    5.58
JE00BG6L7297    5.12
                ... 
GB00B39J2M42    0.50
GB0000667013    0.50
GB00B18S7B29    0.50
GB00BP9LHF23    0.50
GB00B1ZBKY84    0.50
Name: Net Short Position (%), Length: 164, dtype: float64

In [5]:
aapl_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-10-03,23.5,23.5,23.500000,23.5,23.5,662557
2001-10-04,24.0,24.6,24.000000,24.0,24.0,11000
2001-10-05,25.0,25.0,23.000000,24.0,24.0,0
2001-10-08,25.0,25.0,24.000000,24.0,24.0,0
2001-10-09,24.0,24.0,24.000000,24.0,24.0,0
...,...,...,...,...,...,...
2022-12-19,509.5,524.5,504.000000,518.0,518.0,708728
2022-12-20,501.0,519.5,500.500000,507.0,507.0,789353
2022-12-21,510.0,526.5,505.500000,513.5,513.5,549281


In [3]:
import yfinance as yf

for i in range(1000):
    aapl_df = yf.download('ASC.L')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: 

In [88]:
cur_discl[(cur_discl['Position Holder']=='Millennium International Management LP')]

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,Millennium International Management LP,ABCAM PLC,GB00B6774699,0.50,2022-12-12
5,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.90,2022-12-08
46,Millennium International Management LP,BRITISH LAND CO PLC,GB0001367019,0.60,2022-12-12
73,Millennium International Management LP,Dialight PLC,GB0033057794,0.59,2021-06-22
101,Millennium International Management LP,GREAT PORTLAND ESTATES PLC,GB00BF5H9P87,0.92,2022-12-12
240,Millennium International Management LP,SHAFTESBURY PLC,GB0007990962,0.51,2022-12-08
246,Millennium International Management LP,SMITH (DAVID S) HOLDINGS PLC,GB0008220112,0.52,2022-12-08
285,Millennium International Management LP,WIZZ AIR HOLDINGS PLC,JE00BN574F90,0.60,2022-12-08


In [91]:
hist_discl[(hist_discl['Position Holder']=='Millennium International Management LP')&(hist_discl['ISIN']=='GB00BF8Q6K64')]

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
1391,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.80,2022-12-06
1392,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.61,2022-12-02
1393,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.50,2022-11-30
1394,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.60,2022-11-23
1395,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.59,2022-11-18
1396,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.60,2022-11-04
1397,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.50,2022-10-18
1398,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.00,2021-03-19
1399,Millennium International Management LP,abrdn plc,GB00BF8Q6K64,0.53,2021-03-18


In [20]:
total_shorts = cur_discl.groupby('ISIN')['Net Short Position (%)'].sum()
isins = total_shorts.index.to_numpy()

In [25]:
tickers = mapping_df.loc['ticker'] + '.LON'
tickers

GB0030927254     ASC.LON
BMG4593F1389     HSX.LON
ES0177542018     IAG.LON
GB0000055888    DSCV.LON
GB0000667013    AVON.LON
GB0000961622     BBY.LON
Name: ticker, dtype: object